## 1.문제정의)
- "LG"전자에서 2025년 상반기에 새로운 제품을 출시하려고 한다.
- 제품의 키워드는 "음식물 처리기"로 선정
- 타 사제품의 시장조사, 고객의 사용후기 등 데이터 확보 필요

## 2.데이터 수집)
- 소스: 네이버 지식인, 카페, 블로그, 유튜브 댓글, 쇼핑몰 제품 리뷰란 등

## 3.텍스트 데이터 분석)
- 머신러닝, 딥러닝을 활용해서 데이터의 감성분석(긍/부정), 빈도분석 등을 통해 고객의 니즈 분석

### 네이버 지식인 데이터 수집 프로세스
1. 네이버 메인에서 "음식물 처리기" 키워드 검색
2. "지식in" 탭 클릭
3. 게시글 클릭 후 리뷰 데이터 수집
* 지식인 게시글 확보를 위해서 스크롤 내리기 동작 필요
* 기간설정을 통한 지식인 게시글 확보

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

import time
import re

def preprocess_sentence_kr(w):
  w = w.strip()
  w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w)
  w = w.strip()
  return w

In [2]:
# 1.네이버 메인 페이지 요청 후 "음식물 처리기" 검색되도록 구현해보기
driver = wb.Chrome()

In [3]:
url = 'https://www.naver.com'
driver.get(url)

In [4]:
keyword = '그린라이트'
search_input = driver.find_element(By.ID, 'query')
search_input.send_keys(keyword+'\n')

In [5]:
# 2."지식in" 탭 클릭
driver.find_element(By.LINK_TEXT, '지식iN').click()

In [6]:
# 기간설정하는 방법
option_btn = driver.find_element(By.CSS_SELECTOR, '.btn_option._search_option_open_btn')
option_btn.click()

In [7]:
# 기간값을 가진 요소 추출
options = driver.find_elements(By.CSS_SELECTOR, 'li.term a.txt')

options[6].click()

In [8]:
# URL주소의 패턴을 활용한 페이지 요청방법
# %20: 공백을 표현하는 문자
# term: 기간설정 ( 1h, 1w, 1m, 3m, 6m, 1y )
keyword = '그린라이트'
term = '1y'
url2 = f'https://search.naver.com/search.naver?where=kin&query={keyword}&sm=tab_opt&nso=so:r,p:{term},sa:all&answer=0'

print(url2)

https://search.naver.com/search.naver?where=kin&query=그린라이트&sm=tab_opt&nso=so:r,p:1y,sa:all&answer=0


In [9]:
# 3.스크롤 내리기
for i in range(100):
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(1)

In [10]:
# 4.지식인 게시글 URL 수집
#  get_attribute(속성이름)
#  빈 리스트 변수명: href_list 
aTags = driver.find_elements(By.CSS_SELECTOR, 'div.api_save_group + div > a')
len(aTags)

href_list = []

for a in aTags:
    href_list.append(a.get_attribute('href'))

href_list

['https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80208&docId=480050638&enc=utf8&kinsrch_src=pc_tab_kin&qb=6re466aw65287J207Yq4&rank=1&search_sort=0&section=kin.qna_ency_cafe&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=474914297&enc=utf8&kinsrch_src=pc_tab_kin&qb=6re466aw65287J207Yq4&rank=2&search_sort=0&section=kin.qna_ency_cafe&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=476546781&enc=utf8&kinsrch_src=pc_tab_kin&qb=6re466aw65287J207Yq4&rank=3&search_sort=0&section=kin.qna_ency_cafe&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=477928014&enc=utf8&kinsrch_src=pc_tab_kin&qb=6re466aw65287J207Yq4&rank=4&search_sort=0&section=kin.qna_ency_cafe&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=477584558&enc=utf8&kinsrch_src=pc_tab_kin&qb=6re466aw65287J207Yq4&rank=5&search_sort=0&section=kin.qna_ency_cafe&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId

In [ ]:
# 5.확보된 URL 주소로 페이지를 반복적으로 요청
# 6.게시글 내 작성된 리뷰 데이터를 추출하여 텍스트 파일로 저장 
#   파일명: 네이버 지식인 리뷰 데이터.txt
# try:
#   실행문
# except:
#   실행문 or pass
f = open('네이버 지식인 리뷰 데이터.txt','w')

for i in tqdm(range(len(href_list))):
    driver.get(href_list[i])

    time.sleep(1.5)

    try:
        more_btn = driver.find_element(By.ID, 'nextPageButton')
        more_btn.click()
    except:
        pass       

    reviews = driver.find_elements(By.CLASS_NAME, 'se-main-container')

    for j in range(len(reviews)):
        content = preprocess_sentence_kr(reviews[j].text)
        f.write(content)
    
    time.sleep(1.5)

f.close()

 10%|████████                                                                         | 11/110 [00:39<05:44,  3.48s/it]

## Kiwi 라이브러리
- 빠른 속도와 범용적인 성능을 지향하는 지능형 한국어 형태소 분석기
- 웹 텍스트 약 87% | 문어체 텍스트는 약 94% 정확도
- konlpy보다 속도가 조금 더 빠르다는 특징, 설치도 쉬움
- 참고사이트: https://github.com/bab2min/Kiwi?tab=readme-ov-file

In [ ]:
!pip install kiwipiepy

In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [ ]:
# 텍스트 문장 형태소 분석
# form: 실제 형태소
# tag: 품사 태그
# start: 시작하는 인덱스 위치
# len: 형태소 길이
kiwi.tokenize("이번에 고층으로 이사오면서 음식물쓰레기 버리러 나가는게 힘드네요...")

In [ ]:
kiwi.tokenize("이거 정말 정확하게 분석되는 게 맞을까용? 정말 그럴까욬ㅋㅋㅋㅋ")

In [ ]:
# 저장된 텍스트 파일 읽어오기
f = open('./네이버 지식인 리뷰 데이터.txt','r')
text = f.readline()
f.close()

In [ ]:
result = kiwi.tokenize(text)

In [ ]:
result

In [ ]:
noun_list = []

for token in result:
    # 리스트 내 있는 요소와 일치한 값인지 판단
    if token.tag in ['NNG','NNP']: 
        # print(token)
        noun_list.append(token.form)

In [ ]:
noun_list

In [ ]:
# 단어 빈도를 카운팅하는 작업
from collections import Counter

In [ ]:
counter = Counter(noun_list)

In [ ]:
counter

In [ ]:
# 2글자 이상의 명사를 새로운 리스트에 저장
noun_list2 = []

top_100_noun_list = counter.most_common(100)

for noun in top_100_noun_list:
    # print(len(noun[0]))

    if len(noun[0]) > 1:
        noun_list2.append(noun)

In [ ]:
noun_list2

In [ ]:
# 워드클라우드를 그리기 위한 도구
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# mask 이미지를 만들기 위한 도구
from PIL import Image
import numpy as np

In [ ]:
# mask 이미지 만들기
mask_image = np.array(Image.open('./image2.png'))

In [ ]:
wc = WordCloud(
    font_path='C:/Windows/Fonts/malgunbd.ttf',
    background_color='white',
    width=700,
    height=700,
    colormap='spring',
    mask=mask_image
).generate_from_frequencies(dict(noun_list2))

In [ ]:
plt.imshow(wc)
plt.axis('off')
plt.show()